# <center> PATH II </center>

## Unpickling the Noun_Verb_Noun dataframe with 100 articles of tech and automobile articles:

In [1]:
import pickle

filename = r'data\Noun-Verb-Noun.pkl'
getfile = open(filename, 'rb')
kg = pickle.load(getfile)
getfile.close()

## TOC:

1. [Making the tokens column](#token)<br>
2. [Making of Cleaned column by removing cutom stopwords](#stop)<br>
3. [Making of Noun Chunks](#nc)<br>
**  [Better Way](#better) **
4. [Making of edges](#edges)<br>
5. [Making of relations](#relations)<br>
6. [Trying to make verb phrases using Spacy](#verb)<br>
7. [Making of manual check dataframe 1](#manual)<br>
** [Manually identifying the correct triples](#identify)<br>
8. [Checking all 'of' relations in the triples](#of)<br>
9. [Checking all 'in' relations in the triples](#in)<br>
10.[df for Graph](#gr)<br>


In [2]:
kg.head()

,author,content
0,Nicole Wetsman,a case of the new virus spreading rapidly in c...
1,Nicole Wetsman,disease control officials in wuhan the chinese...
2,Nicole Wetsman,scientists think the new virus spreading rapid...
3,Sam Byford,huawei has announced the postponement of a maj...
4,Nicole Wetsman,the world health organization (who) said today...


## Importing neccessary libraries: 

In [3]:
import pandas as pd
import numpy as np
import spacy 
from spacy.matcher import Matcher

pd.set_option('display.max_rows',None)
nlp = spacy.load("en_core_web_sm")

# Matcher class object 
matcher = Matcher(nlp.vocab)

<a id='token'></a>
## Creating a tokens column: 

In [4]:
kg['tokens'] = kg.content.apply(nlp)

In [5]:
kg[:20]

,author,content,tokens
0,Nicole Wetsman,a case of the new virus spreading rapidly in c...,"(a, case, of, the, new, virus, spreading, rapi..."
1,Nicole Wetsman,disease control officials in wuhan the chinese...,"(disease, control, officials, in, wuhan, the, ..."
2,Nicole Wetsman,scientists think the new virus spreading rapid...,"(scientists, think, the, new, virus, spreading..."
3,Sam Byford,huawei has announced the postponement of a maj...,"(huawei, has, announced, the, postponement, of..."
4,Nicole Wetsman,the world health organization (who) said today...,"(the, world, health, organization, (, who, ), ..."
5,Dieter Bohn,welcome back to processor a mostly daily newsl...,"(welcome, back, to, processor, a, mostly, dail..."
6,Dieter Bohn,happy friday to you! i have been reflecting a ...,"(happy, friday, to, you, !, i, have, been, ref..."
7,Nicole Wetsman,your risk of catching the new coronavirus is p...,"(your, risk, of, catching, the, new, coronavir..."
8,Nicole Wetsman,a second us case of the new coronavirus has be...,"(a, second, us, case, of, the, new, coronaviru..."
9,Mary Beth Griggs,as the coronavirus outbreak continues to sprea...,"(as, the, coronavirus, outbreak, continues, to..."


<a id='stop'></a>
## Making of cleaned column by editing spacy stopwords:

In [6]:
kg["cleaned"] = ""

In [7]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
print(len(spacy_stopwords)) ## just copying the stopwords to create a new set

326


In [8]:
stp = {'ten', 'former', 'while', 'thence', 'if', 'being', 'afterwards', 'elsewhere', \
       'when', 'whereby', 'me', 'our', 'did', 'moreover', 'ca', 'besides', 'upon', 'very',\
       'hence', 'put', "'ll", 'seem', 'cannot', 'call', 'therein', 'using', 'he', 'between',\
       'can', 'meanwhile', 'latterly', 'whereafter', 'than', 'mostly', 'during', 'until', \
       'nothing', 'too', 'was', 'seeming', 'across', 'get', 'from', 'both', 'n’t', 'your', \
       'whether', 'six', 'although', 'does', 'beforehand', 'amongst', 'wherein', 'her', 'be',\
       "'ve", 'however', 'perhaps', 'along', 'above', 'name', 'something', '’m', 'another',\
       'everyone', 'any', 'eleven', 'itself', 'two', 'whence', 'were', 'much', 'anyway', \
       'someone', 'quite', 'always', 'seems', 'alone', 'out', 'whenever', 'fifteen', 'forty',\
       'there', 'amount', 'each', 'what', 'have', 'nine', 'thru', 'still', 'really', 'myself',\
       'thereby', 'sometimes', 'nowhere', 'below', 'am', 'nobody', 'those', 'move', 'wherever',\
       'would', 'the', 'see', 'no', 'together', 'do', '‘m', 'here', 'are', 'even', 'neither', \
       'to', 'him', 'must', 'twelve', 'throughout', 'once', 'one', 'among', 'yours', 'thereupon',\
       'but', 'who', 'latter', 'not', 'indeed', 'noone', 'towards', "n't", 'somewhere', 'with',\
       'just', 'less', 'my', 'could', 'becomes', 'these', 'every', 'few', 'his', 'doing', 'else',\
       'they', 'nevertheless', 'hundred', 'on', 'toward', 'rather', '’ll', 'since', 'also',\
       'which', 'hereupon', 'beside', 'made', 'or', 'herself', 'otherwise', 'become', 'will',\
       'is', 'whereas', 'several', 'at', 'about', 'now', 'an', 'under', 'around', 'sixty', \
       'all', 'full', 'others', 'other', 'its', 'behind', 'them', 'many', 'why', 'that', 'give', \
       'mine', 'hereby', 'three', 'without', 'namely', 'everything', 'might', 'never', 'only', \
       'so', '’ve', 'almost', "'m", 'onto', 'whither', 'front', 'us', 'next', 'seemed', 'fifty',\
       'has', 'further', 'whose', 'go', 'after', 'anyhow', 'for', 'serious', 'this', 'none',\
       'regarding', 'before', 'we', 'how', '’s', 'then', 'somehow', 'thus', 'becoming', 'whom',\
       'herein', 'third', 'may', 'against', '‘d', 'except', 'by', 'hereafter', 'into', 'first',\
       'well', 'twenty', 'should', 'unless', 'whole', 'it', 'most', '’re', '’d', 'through',\
       'within', 'had', 'everywhere', 'been', 'keep', 're', 'ours', 'whatever', 'hers', \
       'therefore', 'of', 'sometime', 'beyond', 'as', 'more', 'where', 'take', 'used',\
       'anywhere', 'because', 'own', 'up', 'yet', 'himself', 'i', 'again', 'either', \
       'anything', 'done', 'various', '‘ve', 'last', 'a', '‘s', 'please', 'via', 'same', \
       'she', "'s", 'show', 'eight', 'top', 'side', 'and', 'though', 'their', 'became', \
       'ourselves', 'ever', 'thereafter', 'back', 'say', 'least', 'bottom', 'often', '‘re', \
       'n‘t', 'such', 'off', 'make', 'whoever', 'empty', "'d", 'you', 'whereupon', 'per', 'nor',\
       'enough', 'formerly', 'four', 'part', 'down', 'due', 'some', 'five', 'yourselves', \
       'yourself', 'anyone', "'re", 'over', 'themselves', '‘ll', 'already'}

len(stp)

325

In [9]:
stp.discard('in')   ## discard doesnt throw and error if the given item is not in the set. it will keept the set as is. 
stp.discard('from')
stp.discard('of')
stp.discard('six')

In [10]:
len(stp)

322

In [11]:
## Cleaned column:

In [12]:
for doc in range(len(kg.tokens)):    
    for tok in kg.tokens[doc]:
        if tok not in stp:                                    #.is_stop
            kg.cleaned[doc] += str(tok)+' '

In [13]:
kg.cleaned[0][:100]

'a case of the new virus spreading rapidly in china has been reported in a patient in seattle washing'

<a id='nc'></a>
## Making of Noun Chunks:

In [14]:
noun_chunks_01234 = []


for i in [0,1,2,3,4]:
    for chunk in nlp(kg.cleaned[i]).noun_chunks: #or in [patterns]
        noun_chunks_01234.append(chunk)
        ##add ADP+PROPN+NUM or NUM+PROPN or PROPN+NUM

In [15]:
len(noun_chunks_01234)

730

In [16]:
ent1 = noun_chunks_01234[::2] #even ids of entities

In [17]:
ent2 = noun_chunks_01234[1::2] #odd ids of entities

In [18]:
len(ent1)

365

In [19]:
len(ent2)

365

In [20]:
ent1 = ent1[0:365]

## Identifying the relations:

In [21]:
verbs_01234 = []

vp = ['VERB'] 

for i in [0,1,2,3,4]:
    for each in nlp(kg.cleaned[i]):
        if each.pos_ in vp:
            verbs_01234.append(each)

In [22]:
len(verbs_01234)

394

In [23]:
verbs_01234[:5]

[spreading, reported, reports, returned, monitored]

In [24]:
len(kg.cleaned[0])

3242

In [25]:
kg.cleaned[0][:500]

'a case of the new virus spreading rapidly in china has been reported in a patient in seattle washington reuters reports . the patient had recently returned from china and is clinically healthy but still being monitored . this is the first us case of the virus which was first detected in wuhan a city in central china in late december 2019 . it has already sickened around 300 people and killed six . despite the case report the centers for disease control and prevention ( cdc ) said during a press '

In [26]:
df_01234 = pd.DataFrame({'ent1':ent1[0:365],'verb':verbs_01234[0:365],'ent2':ent2})
df_01234[:20]

,ent1,verb,ent2
0,"(a, case)",spreading,"(the, new, virus)"
1,(china),reported,"(a, patient)"
2,"(seattle, washington, reuters)",reports,"(the, patient)"
3,(china),returned,"(the, first, us, case)"
4,"(the, virus)",monitored,(wuhan)
5,"(a, city)",detected,"(central, china)"
6,"(late, december)",sickened,(it)
7,"(around, 300, people)",killed,"(the, case)"
8,"(the, centers)",report,"(disease, control)"
9,(prevention),said,"((, cdc)"


In [27]:
#df_01234.to_csv("Noun_chunks.csv")

In [28]:
#filename = 'data/Noun-chunks.pkl'
#out = open(filename,'wb')

#pickle.dump(df_012,out)
#out.close()

## Observation:

## The above dataframe is not accurately able to detect the triples. (We getting only 4-5 triples from 25 rows). Thus, next we will look into improving our algorithm to capture more triples. 

## Better Method

<a id='edge'></a>
## Making of edges:

In [29]:
noun_chunks_01234[:4]

[a case, the new virus, china, a patient]

In [30]:
edge_1 = noun_chunks_01234[0::]
print(edge_1[:10])
edge_2 = noun_chunks_01234[1::]
print(edge_2[:10])

[a case, the new virus, china, a patient, seattle washington reuters, the patient, china, the first us case, the virus, wuhan]
[the new virus, china, a patient, seattle washington reuters, the patient, china, the first us case, the virus, wuhan, a city]


In [31]:
edge1 = noun_chunks_01234[0::]
print(edge1[:10])
edge2 = noun_chunks_01234[1::]
print(edge2[:10])

[a case, the new virus, china, a patient, seattle washington reuters, the patient, china, the first us case, the virus, wuhan]
[the new virus, china, a patient, seattle washington reuters, the patient, china, the first us case, the virus, wuhan, a city]


<a id='relations'></a>
## Making of relations:

In [32]:
relations_ = []

vp = ['VERB','ADP'] 

for i in [0,1,2,3,4]:
    for each in nlp(kg.cleaned[i]):
        if each.pos_ in vp:
            relations_.append(each)

In [33]:
print(len(edge_1))
print(len(relations_))
print(len(edge_2))

730
730
729


In [34]:
print(edge_1[:5])
print(relations_[:5])
print(edge_2[:5])

[a case, the new virus, china, a patient, seattle washington reuters]
[of, spreading, in, reported, in]
[the new virus, china, a patient, seattle washington reuters, the patient]


In [35]:
kg.content[0][:300]

'a case of the new virus spreading rapidly in china has been reported in a patient in seattle washington reuters reports. the patient had recently returned from china and is clinically healthy but still being monitored. this is the first us case of the virus which was first detected in wuhan a city i'

<a id='verb'></a>
## Trying to make verb phrases using Spacy Matcher: 

In [36]:
def get_relation(sent):
    doc = nlp(sent)
    verb_list=[]
  # Matcher class object 
    matcher = Matcher(nlp.vocab)

  #AUX-VERB-ADV-ADP: 
    pattern = [{'POS':'AUX','OP':"?"},
               {'POS':'VERB'},   
               {'POS':'ADV','OP':"?"},
               {'POS':'ADP','OP':"?"}] 

    
    matcher.add("matching_1", None, pattern) 

    matches = matcher(doc)
    #print(matches)
    for i in range(len(matches)-1):
        if doc[matches[i][1]] != doc[matches[i+1][1]]:
            span = doc[matches[i][1]:matches[i][2]] 
        #print('----',span)
            verb_list.append(span.text)
    #print(verb_list)
    return(verb_list)

In [37]:
vrb_all=[]
for doc in range(len(kg.content)-1):
    a = get_relation(kg.content[doc])
    #print(a)
    if a != '':
        vrb_all.append(a)

vrb_all = sum(vrb_all,[item for items in vrb_all for item in items])

In [38]:
df_01234_verbph = pd.DataFrame({'ent1':ent1,'verb':vrb_all[0:365],'ent2':ent2})
df_01234_verbph[:10]

,ent1,verb,ent2
0,"(a, case)",spreading rapidly in,"(the, new, virus)"
1,(china),been reported in,"(a, patient)"
2,"(seattle, washington, reuters)",reported in,"(the, patient)"
3,(china),reports,"(the, first, us, case)"
4,"(the, virus)",returned from,(wuhan)
5,"(a, city)",being monitored,"(central, china)"
6,"(late, december)",monitored,(it)
7,"(around, 300, people)",detected in,"(the, case)"
8,"(the, centers)",sickened around,"(disease, control)"
9,(prevention),killed,"((, cdc)"


In [39]:
doc = nlp(kg.content[0])
matcher = Matcher(nlp.vocab)

# This is a triple and can define a relationship
pattern1 = [{'POS': 'NOUN','OP':'+'},
            {'POS': 'ADP','OP':'?'},
            {'POS': 'NOUN','OP':'+'}] 

matcher.add('Match1', None, pattern1) # add the pattern to the matcher 
matches = matcher(doc)
#print(matches)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

2356062832825383259 Match1 91 93 press briefing
2356062832825383259 Match1 121 124 family of viruses
2356062832825383259 Match1 151 153 coronavirus expert
2356062832825383259 Match1 227 229 hospital staff
2356062832825383259 Match1 252 254 health screenings
2356062832825383259 Match1 319 321 health officials
2356062832825383259 Match1 331 334 threats from coronaviruses
2356062832825383259 Match1 390 393 sars for example
2356062832825383259 Match1 405 407 seafood market
2356062832825383259 Match1 450 452 health authorities
2356062832825383259 Match1 478 480 animal reservoir
2356062832825383259 Match1 498 501 human to human
2356062832825383259 Match1 541 543 health emergency


## Conclusion:<br>We would like to add the above findings into our dataset but this is not something which we can use to build datasets. 

<a id='manual'></a>
## Making of manual checking dataframe:

In [40]:
data = {'N1':edge_1[0:729],'relations':relations_[0:729],'N2':edge_2[0:729]}

manual_check_1 = pd.DataFrame(data)
manual_check_1[:20]

,N1,relations,N2
0,"(a, case)",of,"(the, new, virus)"
1,"(the, new, virus)",spreading,(china)
2,(china),in,"(a, patient)"
3,"(a, patient)",reported,"(seattle, washington, reuters)"
4,"(seattle, washington, reuters)",in,"(the, patient)"
5,"(the, patient)",in,(china)
6,(china),reports,"(the, first, us, case)"
7,"(the, first, us, case)",returned,"(the, virus)"
8,"(the, virus)",from,(wuhan)
9,(wuhan),monitored,"(a, city)"


In [41]:
#manual_check_1.to_csv('manual_check_1.csv')

## EDA on the new triples dataframe to understand its accuracy:

In [42]:
manual_check_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   N1         729 non-null    object
 1   relations  729 non-null    object
 2   N2         729 non-null    object
dtypes: object(3)
memory usage: 17.2+ KB


In [43]:
manual_check_1.N1 = manual_check_1.N1.astype('str')
manual_check_1.relations = manual_check_1.relations.astype('str')
manual_check_1.N2 = manual_check_1.N2.astype('str')

In [44]:
manual_check_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   N1         729 non-null    object
 1   relations  729 non-null    object
 2   N2         729 non-null    object
dtypes: object(3)
memory usage: 17.2+ KB


<a id='identify'></a>
## Manually identifying the correct triples:<br>
1. [Checking all 'of' relations in the triples](#of)<br>
2. [Checking all 'in' relations in the triples](#in)<br>

<a id='of'></a>
## 'of' :

In [45]:
of_df = manual_check_1[manual_check_1.relations=='of'].reset_index()
len(of_df)

69

In [46]:
len(of_df[0::2])   ## -> 42% (15/35) accuracy

35

In [47]:
(42+44)/2

43.0

In [48]:
len(of_df[1::2]) ## 15/34 44% correct triples 

34

In [49]:
print('Total accuracy for triples with \'of\' relation:',round(30/69,3),'%') 

Total accuracy for triples with 'of' relation: 0.435 %


In [50]:
manual_check_1.iloc[679]

N1           this kind
relations           of
N2           emergency
Name: 679, dtype: object

<a id='in'></a>
## 'in' :

In [51]:
in_df = manual_check_1[manual_check_1.relations=='in'].reset_index()
len(in_df)

68

In [52]:
len(in_df[0::2])   ## -> 26.47% (9/34) accuracy

34

In [53]:
len(in_df[1::2]) ## 19/34 55.88% correct triples 

34

In [54]:
print('Total accuracy for triples with \'in\' relation:',round(28/68,3),'%')

Total accuracy for triples with 'in' relation: 0.412 %


In [55]:
69+68

137

In [56]:
correct = [679,706,727,347,444,524,528,565,575,0,37,103,217,299,307,10,36,\
          229,318,406,439,492,515,526,564,675,683,704,707,715,10,36,229,318,406,439,\
          492,515,526,564,675,683,704,707,715,5,165,374,388,392,442,530,688,712,4,\
          87,161,322,328,336,345,373,378,383,389,395,432,472,534,556,600,642,685,717]
print(len(correct))

for each in [0,1,3,9,12,13,16,18,19,20,23,24,27,28,29,30,31,32,33,37,39,40,42,44,45,46,49,\
            50,51,52,53,56,57,59,61,64,67,68,71,74,76, 84, 89, 100, 105, 110, 113, 114, 116,\
            117,124,127,141,144,156,166,178,179,184,186,187,188,189,190,193]:
        
        if each in correct:
            pass
        else:
            correct.append(each)
print(len(correct))
            
df_correct_1 = [manual_check_1.iloc[each] for each in correct]   # -- 1

df_correct_1 = pd.DataFrame(df_correct_1).reset_index(drop=True) 
df_correct_1.head()

74
137


,N1,relations,N2
0,this kind,of,emergency
1,the aggressive efforts,of,chinese officials
2,any additional restrictions,of,travel
3,kristian andersen director,of,infectious disease genomics
4,the full genetic sequence,of,the virus


In [86]:
len(df_correct_1)

137

In [87]:
len(manual_check_1)

660

In [88]:
if 'in' in manual_check_1.relations: ##to avoid errors, we know it should be 661
    manual_check_1.drop(manual_check_1[manual_check_1.relations=='in'].index,inplace=True)

In [89]:
len(manual_check_1)

660

In [90]:
if len(manual_check_1)>592: ##to avoid errors, we know it should be 592
    manual_check_1.drop(manual_check_1[manual_check_1.relations=='of'].index,inplace=True)

In [91]:
len(manual_check_1)

660

In [92]:
manual_check_1[:10]

,N1,relations,N2
1,the new virus,spreading,china
2,china,in,a patient
3,a patient,reported,seattle washington reuters
4,seattle washington reuters,in,the patient
5,the patient,in,china
6,china,reports,the first us case
7,the first us case,returned,the virus
8,the virus,from,wuhan
9,wuhan,monitored,a city
11,central china,detected,late december


<a id='#gr'></a>
## Making the graph on df_correct_1 triples

In [93]:
len(df_correct_1.N1)

137

In [94]:
len(df_correct_1.N1.unique()) ## We have 57 unique values

105

In [95]:
all_nodes = np.append(df_correct_1.N1,df_correct_1.N2)
len(all_nodes)
len(np.unique(all_nodes))

163

In [96]:
## Making a dictionary with the unique values:
all_nodes = {value:key for (key,value) in dict(enumerate(np.unique(all_nodes),0)).items()}

In [97]:
## make a new column with the encoded keys for N1 column: 
df_correct_1['key_n1'] = df_correct_1['N1'].copy(deep=True).map(all_nodes)

In [98]:
sum(df_correct_1.key_n1.isna()) ## checking if there are any NaN values in the key_n1 column 

0

In [99]:
## make a new column with the encoded keys for N2 column: 
df_correct_1['key_n2'] = df_correct_1['N2'].copy(deep=True).map(all_nodes)

In [100]:
sum(df_correct_1.key_n2.isna()) ## checking if there are any NaN values in the key_n1 column

0

In [101]:
df_correct_1[:5]

,N1,relations,N2,key_n1,key_n2
0,this kind,of,emergency,147,40
1,the aggressive efforts,of,chinese officials,93,33
2,any additional restrictions,of,travel,23,155
3,kristian andersen director,of,infectious disease genomics,62,57
4,the full genetic sequence,of,the virus,109,141


In [102]:
## Creating keys to get all the keys from both the entity columns:
keys = np.unique(np.append(df_correct_1.key_n1,df_correct_1.key_n2))
keys[:3]

array([0, 1, 2], dtype=int64)

In [103]:
## Corresponding names of the keys:
name = np.unique(np.append(df_correct_1.N1,df_correct_1.N2))
name[:4]

array(['( cdc', '17 people', '2019 ) zika', 'a case'], dtype=object)

In [104]:
type(all_nodes)

dict

In [105]:
## Creating the nodes dataframe from unique values in the data:
dfNodes = pd.DataFrame.from_dict(all_nodes,orient='index')

In [106]:
dfNodes = dfNodes.reset_index().rename(columns={'index':'name',0:'keys'})
dfNodes[:4]

,name,keys
0,( cdc,0
1,17 people,1
2,2019 ) zika,2
3,a case,3


In [107]:
dfNodes = dfNodes.reindex(columns=['keys','name'])

## Saving the links and the nodes data to csv:

In [109]:
df_correct_1.to_csv(r'data\Links.csv')

In [110]:
dfNodes.to_csv(r'data\Nodes.csv')

## Pickling the links and nodes dataframes:

In [83]:
filename = 'data/Links.pkl'
out = open(filename,'wb')

pickle.dump(df_correct_1,out)
out.close()

In [84]:
filename = 'data/Nodes.pkl'
out = open(filename,'wb')

pickle.dump(dfNodes,out)
out.close()

## Conclusion: 

## We prepared the Links and Nodes csv files for our NetworkD3 graph R script next. 